In [435]:
import os
import matplotlib.pyplot as plt
import numpy as np
plt.ioff();
try:
    maindir
except NameError:
    maindir = os.path.dirname(os.getcwd())
for module in os.listdir(maindir + '\\testing_modules'):
    %run -i "{maindir}\\testing_modules\\{module}"
base = os.path.dirname(os.path.dirname(maindir)) + '\\'
plt.ion();
np.set_printoptions(suppress=True)

In [2]:
%matplotlib qt
import skimage.io
from scipy.ndimage import center_of_mass, gaussian_filter, median_filter
from skimage.measure import label, regionprops, find_contours
from skimage import filters
from skimage import restoration
import pandas as pd
from scipy.optimize import curve_fit
from scipy.signal import find_peaks, correlate, convolve2d
from skimage.feature import peak_local_max
from skimage.segmentation import watershed, morphological_geodesic_active_contour, random_walker, expand_labels
import pyFAI
import pyFAI.azimuthalIntegrator as ai

In [3]:
filedir = '''DOE_SCGSR\\Data\\221128 Beamtime\\maps\\'''
map_filename = '''scan134664-002.tif'''

map_img = skimage.io.imread(base + filedir + map_filename)
flat_field_est = np.median(map_img, axis=(0, 1))
mask_img = np.bool_(skimage.io.imread(maindir + '''\\testing_resources\\refined_merlin_mask.tif'''))
mask_img[mask_img] = np.nan
flat_img = skimage.io.imread(base + filedir + '''merlin_flat_field.tif''')
print(map_img.shape)

(71, 71, 512, 512)


In [436]:
ai = pyFAI.load(maindir + '\\testing_resources\\ZnO_indexing_v3.poni')
ai.energy = 15
tth_resolution = 0.02 # Degrees
chi_resolution = 0.05 # Degrees

res = ai.integrate2d_ng(map_img.reshape(map_img.shape[0] * map_img.shape[1], 512, 512)[0], 1024, 360, unit='2th_deg', mask=~mask_img, flat=flat_img)
tth, chi = res[1], res[2]
tth_num = int(np.abs(np.max(tth) - np.min(tth)) // tth_resolution)
chi_num = int(np.abs(np.max(chi) - np.min(chi)) // chi_resolution)
# The bounds of interpolation should be limited by the intrument resolution AND the original image size. Should not make up too much data

# There is a better way I could do this...
res = ai.integrate2d_ng(map_img.reshape(map_img.shape[0] * map_img.shape[1], 512, 512)[0], tth_num, chi_num, unit='2th_deg', mask=~mask_img, flat=flat_img)
tth, chi = res[1], res[2]
extent = [tth[0], tth[-1], chi[0], chi[-1]]

In [ ]:
mult = 1
sig_thresh = mult * np.std(np.multiply(flat_img, mask_img))

j, i = 15, 63 
j, i = 46, 24
#test_img = interpolate_merlin_mask(np.multiply((np.copy(map_img[i, j]) - flat_img), mask_img))
#test_img = interpolate_merlin_mask(np.multiply((np.copy(calibrated_map[i, j]) - flat_img), mask_img))
#fixed_img = find_outlier_pixels(test_img, size=2, tolerance=3, significance=5 * np.std(np.multiply(flat_img, mask_img)))

test_img = interpolate_merlin_mask(np.multiply((np.copy(map_img[i, j]) - flat_img), mask_img))
fixed_img = find_outlier_pixels(test_img, size=2, tolerance=3, significance=5 * np.std(np.multiply(flat_img, mask_img)))
res = ai.integrate2d_ng(fixed_img, tth_num, chi_num, unit='2th_deg')
fixed_img = res[0]

fig, ax = plt.subplots(1, 2, figsize=(10, 5), dpi=200, sharex=True, sharey=True)

im = ax[0].imshow(fixed_img, extent=extent, vmin=0, vmax=100)
fig.colorbar(im, ax=ax[0])
ax[0].set_xlabel('Scattering Angle, 2θ [°]')
ax[0].set_ylabel('Azimuthal Angle, χ [°]')
proc_img = gaussian_filter(fixed_img, sigma=3)
proc_img[proc_img < sig_thresh] = np.nan
im = ax[1].imshow(proc_img, extent=extent, vmin=0, vmax=100)
fig.colorbar(im, ax=ax[1])
ax[1].set_xlabel('Scattering Angle, 2θ [°]')
ax[1].set_facecolor('k')

plt.show()

In [437]:
# The Whole Thing
mult = 1
sig_thresh = mult * np.std(np.multiply(flat_img, mask_img))

j, i = 29, 5
test_img = interpolate_merlin_mask(np.multiply((np.copy(map_img[i, j]) - flat_img), mask_img))
fixed_img = find_outlier_pixels(test_img, size=2, tolerance=3, significance=5 * np.std(np.multiply(flat_img, mask_img)))
res = ai.integrate2d_ng(fixed_img, tth_num, chi_num, unit='2th_deg')
fixed_img = res[0]

# Finding significant blobs
blob_df = find_blobs(fixed_img, sig_thresh, plotme=False, listme=False)

# Fitting all blobs with peak functions
blob_df = fit_blobs(fixed_img, sig_thresh, blob_df, GaussianFunctions(), plotme=True)

# Find spots within blobs
blob_df = find_blob_spots(fixed_img, blob_df, plotme=False)

Peak fitting exceeds predicted behavior for blob 3. It may not be significant.


In [427]:
empty_lst = []
bad_lst = [1, 2, 3, np.all]

~np.isnan(bad_lst[:-1])

array([ True,  True,  True])

In [433]:
if popt != np.nan and np.all(~np.isnan(popt[:-2])):
    pass

TypeError: 'float' object is not subscriptable

In [439]:
popt

[nan]

In [452]:
# Adaptive peak fitting within blobs
# TODO: Allow for peak addition with residual?
PeakModel = LorentzianFunctions()
x_coords, y_coords = np.meshgrid(tth, chi[::-1])

fig, ax = plt.subplots(1, 1, figsize=(6, 5), dpi=200)
im = ax.imshow(fixed_img, extent=extent, vmin=0)
fig.colorbar(im, ax=ax)

spot_fits = []
for blob_num in range(len(blob_df)):
    
    z_fit = blob_df['value'][blob_num][blob_df['mask'][blob_num]]
    x_fit = x_coords[blob_df['mask'][blob_num]]
    y_fit = y_coords[blob_df['mask'][blob_num]]

    # Only works for Gaussian and Lorentzian. Create function to generate these...
    p0 = []
    for i, spot in enumerate(blob_df['spots'][blob_num]):
        p0.append(blob_df['spots_intensity'][blob_num][i]) # amp
        p0.append(spot[0]) # x0
        p0.append(spot[1]) # y0
        p0.append(0.05) # sigma_x
        p0.append(0.05) # sigma_y
        p0.append(0) # theta


    ax.plot(*blob_df['contour'][blob_num], c='r', lw=0.5)

    bounds = generate_bounds(p0, PeakModel.func_2d, tth_step=None, chi_step=None)

    adaptive_fitting = True
    while adaptive_fitting:
        peaks_discounted = False
        popt, pcov = curve_fit(PeakModel.multi_2d, np.vstack([x_fit, y_fit]), z_fit, p0=p0, bounds=bounds)
        r_squared = compute_r_squared(z_fit, PeakModel.multi_2d((x_fit, y_fit), *popt))
        
        old_p0 = p0.copy()
        for i in range(len(popt) // 6):
            if not qualify_gaussian_2d_fit(0.5, 4 * np.std(fixed_img), old_p0[6 * i:6 * i + 6], popt[6 * i:6 * i + 6]):
                print(f'Spot {i} discounted.')
                #print(old_p0[6 * i:6 * i + 6])
                #print(popt[6 * i:6 * i + 6])
                del(p0[6 * i:6 * i + 6])
                del(bounds[0][6 * i:6 * i + 6])
                del(bounds[1][6 * i:6 * i + 6])
                #print(len(p0) // 7)
                peaks_discounted = True
        if len(p0) == 0:
            adaptive_fitting = False
            print(f'Eliminated all local maxima. Defaulting to full blob fit for {blob_num}')
            popt = blob_df['fit_parameters'][blob_num]
            if len(popt) > 1:
                popt = popt[:-2]
            else:
                print('No blob fit either!. Blob appears to be insignificant. Consider removing?')
        else:
            adaptive_fitting = peaks_discounted
        if adaptive_fitting:
            print('Refitting...')
        elif np.all(~np.isnan(popt[:-2])) and (len(popt) > 1):
            print(f'Fitting successful for {len(popt) // 6} spot(s) within blob {blob_num}!')
            spot_fits.append(popt)

    if np.all(~np.isnan(popt[:-2])) and (len(popt) > 1):       
        fit_data = PeakModel.multi_2d((x_coords.ravel(), y_coords.ravel()), *popt)
        ax.contour(x_coords, y_coords, fit_data.reshape(*fixed_img.shape), np.linspace(sig_thresh, popt[0], 5), colors='w', linewidths=0.5)
        ax.scatter(np.asarray(popt).reshape(len(popt) // 6, 6)[:, 1], np.asarray(popt).reshape(len(popt) // 6, 6)[:, 2], s=1, c='r')

        props = dict(boxstyle='round', facecolor='white', alpha=0.5)
        arrow_props = dict(facecolor='black', shrink=0.1, width=1, headwidth=5, headlength=5)
        ax.annotate(f'R² = {r_squared:.2f}', xy=(popt[1], popt[2]), xytext=(popt[1] + 1, popt[2] + 1),
                    arrowprops=arrow_props, bbox=props, fontsize=8)
    plt.show()

Fitting successful for 5 spot(s) within blob 0!
Fitting successful for 2 spot(s) within blob 1!
Fitting successful for 2 spot(s) within blob 2!
Spot 0 discounted.
Eliminated all local maxima. Defaulting to full blob fit for 3
No blob fit either!. Blob appears to be insignificant. Consider removing?


In [451]:
np.array([item for sublist in spot_fits for item in sublist]).reshape(9, 6)

array([[158.23638826,  60.73308523, -85.84311264,   0.00993016,
          0.04452037,   4.83919889],
       [123.81929803,  60.44741342, -86.34205735,   0.01292024,
          0.08094949,   4.61036958],
       [130.56707061,  60.21221127, -86.09751025,   0.01939581,
          0.03132624, -10.84981026],
       [120.03375032,  60.73257293, -86.23933602,   0.00902248,
          0.01322185,   5.79620985],
       [ 26.24506446,  60.63250835, -86.97458931,   0.00832716,
          0.25994256, -18.8388649 ],
       [213.58215231,  60.25226294, -80.70426859,   0.00660661,
          0.01613719,  26.70347516],
       [ 31.63877033,  60.22946027, -79.58241809,   0.02530064,
          0.03820792, -35.96351221],
       [ 56.02997956,  60.51853976, -92.89884021,   0.00278035,
          0.0104529 ,  -8.96130386],
       [ 32.34590471,  60.46973844, -93.32130336,   0.00374779,
          0.01686082,  -6.77630123]])

In [5]:
# Integrated blob intensity for the full map.
'''mult = 0.75
sig_thresh = mult * np.std(np.multiply(flat_img, mask_img))

plot_profile = []

for i, img in enumerate(map_img.reshape(map_img.shape[0] * map_img.shape[1], *map_img.shape[2:])):
    test_img = interpolate_merlin_mask(np.multiply((np.copy(img) - flat_img), mask_img))
    fixed_img = find_outlier_pixels(test_img, size=2, tolerance=3, significance=5 * np.std(np.multiply(flat_img, mask_img)))
    proc_img = gaussian_filter(fixed_img, sigma=3)
    plot_profile.append(np.sum(fixed_img[proc_img > sig_thresh]))

plot_profile = np.asarray(plot_profile).reshape(*map_img.shape[:2])'''

In [131]:
# Determine and plot residual image, within found blobs...
'''fit_img = np.zeros_like(fixed_img)

for i in range(len(blob_df)):
    if not np.any(np.isnan(blob_df['gaussian_fit'][i])):
        blob_fit = gaussian_2d((x_coords, y_coords), *blob_df['gaussian_fit'][i][:-1]) - blob_df['gaussian_fit'][i][3]
        fit_img += blob_fit

fit_img += np.mean(np.asarray([x for x in blob_df['gaussian_fit'].to_list() if np.any(~np.isnan(x))])[:, 3])
res_img = fixed_img * ~bkg_mask - fit_img * ~bkg_mask

fig, ax = plt.subplots(1, 3, figsize=(10, 3), dpi=200, sharex=True, sharey=True)
im = ax[0].imshow(fixed_img, vmin=0, vmax=100)
fig.colorbar(im , ax=ax[0], shrink=0.6)
ax[0].set_title('Experiment')
im = ax[1].imshow(fit_img, vmin=0, vmax=100)
fig.colorbar(im , ax=ax[1], shrink=0.6)
ax[1].set_title('Blob Fit')
im = ax[2].imshow(res_img, vmin=-50, vmax=50, cmap='bwr')
fig.colorbar(im , ax=ax[2], shrink=0.6)
ax[2].set_title('Residual')

for i in range(len(blob_df)):
    ax[0].plot(blob_df['contour'][i].T[1], blob_df['contour'][i].T[0], c='r', lw=0.5)
    ax[1].plot(blob_df['contour'][i].T[1], blob_df['contour'][i].T[0], c='r', lw=0.5)
    ax[2].plot(blob_df['contour'][i].T[1], blob_df['contour'][i].T[0], c='r', lw=0.5)

plt.show()'''

"fit_img = np.zeros_like(fixed_img)\n\nfor i in range(len(blob_df)):\n    if not np.any(np.isnan(blob_df['gaussian_fit'][i])):\n        blob_fit = gaussian_2d((x_coords, y_coords), *blob_df['gaussian_fit'][i][:-1]) - blob_df['gaussian_fit'][i][3]\n        fit_img += blob_fit\n\nfit_img += np.mean(np.asarray([x for x in blob_df['gaussian_fit'].to_list() if np.any(~np.isnan(x))])[:, 3])\nres_img = fixed_img * ~bkg_mask - fit_img * ~bkg_mask\n\nfig, ax = plt.subplots(1, 3, figsize=(10, 3), dpi=200, sharex=True, sharey=True)\nim = ax[0].imshow(fixed_img, vmin=0, vmax=100)\nfig.colorbar(im , ax=ax[0], shrink=0.6)\nax[0].set_title('Experiment')\nim = ax[1].imshow(fit_img, vmin=0, vmax=100)\nfig.colorbar(im , ax=ax[1], shrink=0.6)\nax[1].set_title('Blob Fit')\nim = ax[2].imshow(res_img, vmin=-50, vmax=50, cmap='bwr')\nfig.colorbar(im , ax=ax[2], shrink=0.6)\nax[2].set_title('Residual')\n\nfor i in range(len(blob_df)):\n    ax[0].plot(blob_df['contour'][i].T[1], blob_df['contour'][i].T[0], c='

In [15]:
blob_num = 1

#filt_img = median_filter(fixed_img * blob_df['mask'][1], size=2)
filt_img = gaussian_filter(median_filter(fixed_img * blob_df['mask'][blob_num], size=2), sigma=0.5)
#filt_img = -filters.sobel(gaussian_filter(fixed_img * blob_df['mask'][1], sigma=1.5))
#filt_img = filters.roberts(filters.roberts(gaussian_filter(fixed_img * blob_df['mask'][1], sigma=1)))

x_img_lst = np.linspace(0, tth_num - 1, tth_num)
y_img_lst = np.linspace(0, chi_num - 1, chi_num)
x_img, y_img = np.meshgrid(x_img_lst, y_img_lst)

blob_bbox = [int(np.min(x_img[blob_df['mask'][blob_num]])), int(np.max(x_img[blob_df['mask'][blob_num]])) + 1,
             int(np.min(y_img[blob_df['mask'][blob_num]])), int(np.max(y_img[blob_df['mask'][blob_num]])) + 1]

fig, ax = plt.subplots(1, 2, figsize=(6, 5), dpi=200, sharex=True, sharey=True)
ax[0].imshow(fixed_img, extent=extent, vmin=0)
ax[1].imshow(filt_img, extent=extent, vmin=0)
# Peak search in blobl bounding box to reduce computational time
local_spots = peak_local_max(filt_img[blob_bbox[2]:blob_bbox[3], blob_bbox[0]:blob_bbox[1]], min_distance=3, threshold_rel=0.15)
if local_spots.shape[0] == 0:
    print('No local maxima found. Using total blob maximum.')
    local_spots = np.array([blob_df['max_coords'][blob_num]])[0]
    local_max = np.max(blob_df['value'][blob_num])
elif local_spots.shape[0] > 1: # discounts any insignificant peaks if more than one local maxima found
    significant_spots = [filt_img[blob_bbox[2]:blob_bbox[3], blob_bbox[0]:blob_bbox[1]][*spot] > np.std(fixed_img) for spot in local_spots]
    local_spots = local_spots[significant_spots]
local_max = [fixed_img[blob_bbox[2]:blob_bbox[3], blob_bbox[0]:blob_bbox[1]][*spot] for spot in local_spots]
local_spots = np.array([[x_coords[blob_bbox[2]:blob_bbox[3], blob_bbox[0]:blob_bbox[1]][*spot],
                y_coords[blob_bbox[2]:blob_bbox[3], blob_bbox[0]:blob_bbox[1]][*spot]] for spot in local_spots])
for axi in ax:
    axi.scatter(local_spots[:, 0], local_spots[:, 1], c='r', s=1)
    axi.plot(*blob_df['contour'][blob_num], c='r', lw=0.5)

plt.show()
print(f'Found {len(local_spots)} possible spots within blob {blob_num}.')

Found 2 possible spots within blob 1.


In [133]:
# Alternate method for finding local maxima
'''filt_img = -filters.sobel(gaussian_filter(fixed_img * blob_df['mask'][1], sigma=1))

local_maxima = []
for i, j in zip(y_coords[blob_bbox[2]:blob_bbox[3], blob_bbox[0]:blob_bbox[1]].flatten(), x_coords[blob_bbox[2]:blob_bbox[3], blob_bbox[0]:blob_bbox[1]].flatten()):
    i, j = int(i), int(j)
    condition = np.all([(filt_img[i, j] > filt_img[i + 1, j]),
                        (filt_img[i, j] > filt_img[i - 1, j]),
                        (filt_img[i, j] > filt_img[i, j + 1]),
                        (filt_img[i, j] > filt_img[i, j - 1])])
    if condition:
        local_maxima.append([i, j])
local_maxima = np.asarray(local_maxima)

fig, ax = plt.subplots(1, 2, figsize=(6, 5), dpi=200, sharex=True, sharey=True)
ax[0].imshow(fixed_img, vmin=0, vmax=100)
ax[1].imshow(filt_img)
for axi in ax:
    axi.scatter(x_coords[local_maxima[:, 0], local_maxima[:, 1]], y_coords[local_maxima[:, 0], local_maxima[:, 1]], c='r', s=1)

plt.show()''';

In [134]:
# Fitting peaks within a blob
'''z_fit = blob_df['value'][blob_num][blob_bbox[2]:blob_bbox[3], blob_bbox[0]:blob_bbox[1]]
x_fit = x_coords[blob_bbox[2]:blob_bbox[3], blob_bbox[0]:blob_bbox[1]].ravel()
y_fit = y_coords[blob_bbox[2]:blob_bbox[3], blob_bbox[0]:blob_bbox[1]].ravel()

p0 = []
for i, peak in enumerate(local_peaks):
    if i in [1, 2]:
        continue
    p0.append(z_fit[peak[0], peak[1]]) # amp
    p0.append(peak[0] + blob_bbox[2]) # y0
    p0.append(peak[1] + blob_bbox[0]) # x0
    p0.append(0) # z0
    p0.append(2) # sigma_x
    p0.append(2) # sigma_y
    p0.append(0) # theta

z_fit = z_fit.ravel()

fig, ax = plt.subplots(1, 1, figsize=(6, 5), dpi=200)
im = ax.imshow(fixed_img, vmin=0)
fig.colorbar(im, ax=ax)
for i in range(len(blob_df)):
    ax.plot(blob_df['contour'][i].T[1], blob_df['contour'].T[0], c='r', lw=0.5)

def fit_func(xy, *args):
    return multi_peak_fitting(xy, gaussian_2d, *args)

popt, pcov = curve_fit(fit_func, np.vstack([x_fit, y_fit]), z_fit, p0=p0)
r_squared = compute_r_squared(z_fit, fit_func((x_fit, y_fit), *popt))

for i in range(len(popt) // 7):
    
    if not qualify_gaussian_2d_fit(r_squared, 3 * np.std(fixed_img), p0[7 * i:7*i + 7], popt[7 * i:7*i + 7]):
        print(p0[7 * i:7*i + 7])
        print(popt[7 * i:7*i + 7])
        print(f'Peak {i} should be discounted and refit')

fit_data = fit_func((x_coords.ravel(), y_coords.ravel()), *popt)


ax.contour(x_coords, y_coords, fit_data.reshape(*fixed_img.shape), 10, colors='w', linewidths=0.5)
ax.scatter(np.asarray(popt).reshape(len(popt) // 7, 7)[:, 2], np.asarray(popt).reshape(len(popt) // 7, 7)[:, 1], s=1, c='r')

props = dict(boxstyle='round', facecolor='white', alpha=0.5)
arrow_props = dict(facecolor='black', shrink=0.1, width=1, headwidth=5, headlength=5)
ax.annotate(f'R² = {r_squared:.2f}', xy=(popt[2], popt[1]), xytext=(popt[2] + 20, popt[1] + 20),
            arrowprops=arrow_props, bbox=props, fontsize=8)
plt.show()'''

"z_fit = blob_df['value'][blob_num][blob_bbox[2]:blob_bbox[3], blob_bbox[0]:blob_bbox[1]]\nx_fit = x_coords[blob_bbox[2]:blob_bbox[3], blob_bbox[0]:blob_bbox[1]].ravel()\ny_fit = y_coords[blob_bbox[2]:blob_bbox[3], blob_bbox[0]:blob_bbox[1]].ravel()\n\np0 = []\nfor i, peak in enumerate(local_peaks):\n    if i in [1, 2]:\n        continue\n    p0.append(z_fit[peak[0], peak[1]]) # amp\n    p0.append(peak[0] + blob_bbox[2]) # y0\n    p0.append(peak[1] + blob_bbox[0]) # x0\n    p0.append(0) # z0\n    p0.append(2) # sigma_x\n    p0.append(2) # sigma_y\n    p0.append(0) # theta\n\nz_fit = z_fit.ravel()\n\nfig, ax = plt.subplots(1, 1, figsize=(6, 5), dpi=200)\nim = ax.imshow(fixed_img, vmin=0)\nfig.colorbar(im, ax=ax)\nfor i in range(len(blob_df)):\n    ax.plot(blob_df['contour'][i].T[1], blob_df['contour'].T[0], c='r', lw=0.5)\n\ndef fit_func(xy, *args):\n    return multi_peak_fitting(xy, gaussian_2d, *args)\n\npopt, pcov = curve_fit(fit_func, np.vstack([x_fit, y_fit]), z_fit, p0=p0)\nr_squa

In [16]:
# Adaptive peak fitting within blobs
# TODO: Allow for peak addition with residual?
PeakModel = LorentzianFunctions()
z_fit = blob_df['value'][blob_num][blob_bbox[2]:blob_bbox[3], blob_bbox[0]:blob_bbox[1]].ravel()
x_fit = x_coords[blob_bbox[2]:blob_bbox[3], blob_bbox[0]:blob_bbox[1]].ravel()
y_fit = y_coords[blob_bbox[2]:blob_bbox[3], blob_bbox[0]:blob_bbox[1]].ravel()

p0 = []
for i, peak in enumerate(local_spots):
    p0.append(local_max[i]) # amp
    p0.append(peak[0]) # y0
    p0.append(peak[1]) # x0
    p0.append(0.05) # sigma_x
    p0.append(0.05) # sigma_y
    p0.append(0) # theta

fig, ax = plt.subplots(1, 1, figsize=(6, 5), dpi=200)
im = ax.imshow(fixed_img, extent=extent, vmin=0)
fig.colorbar(im, ax=ax)
ax.plot(*blob_df['contour'][blob_num], c='r', lw=0.5)

bounds = generate_bounds(p0, PeakModel.func_2d, tth_step=None, chi_step=None)

adaptive_fitting = True
while adaptive_fitting:
    popt, pcov = curve_fit(PeakModel.multi_2d, np.vstack([x_fit, y_fit]), z_fit, p0=p0, bounds=bounds)
    r_squared = compute_r_squared(z_fit, PeakModel.multi_2d((x_fit, y_fit), *popt))

    peaks_discounted = False
    old_p0 = p0.copy()
    for i in range(len(popt) // 6):
        if not qualify_gaussian_2d_fit(0.5, 4 * np.std(fixed_img), old_p0[6 * i:6 * i + 6], popt[6 * i:6 * i + 6]):
            print(f'Spot {i} discounted.')
            print(old_p0[6 * i:6 * i + 6])
            print(popt[6 * i:6 * i + 6])
            del(p0[6 * i:6 * i + 6])
            del(bounds[0][6 * i:6 * i + 6])
            del(bounds[1][6 * i:6 * i + 6])
            #print(len(p0) // 7)
            peaks_discounted = True
    if len(p0) == 0:
        adaptive_fitting = False
        print('Eliminated all local maxima. Defaulting to full blob fit.')
        popt = blob_df['gaussian_fit'][blob_num]
        if np.all(~np.isnan(popt)):
            popt = popt[:-1]
        else:
            print('No blob fit either!. Blob appears to be insignificant. Consider removing?')
    else:
        adaptive_fitting = peaks_discounted
    if adaptive_fitting:
        print('Refitting...')
    elif np.all(~np.isnan(popt)):
        print(f'Fitting successful for {len(popt) // 6} spot(s) within blob {blob_num}!')

if np.all(~np.isnan(popt)):       
    fit_data = PeakModel.multi_2d((x_coords.ravel(), y_coords.ravel()), *popt)
    ax.contour(x_coords, y_coords, fit_data.reshape(*fixed_img.shape), np.linspace(sig_thresh, popt[0], 5), colors='w', linewidths=0.5)
    ax.scatter(np.asarray(popt).reshape(len(popt) // 6, 6)[:, 1], np.asarray(popt).reshape(len(popt) // 6, 6)[:, 2], s=1, c='r')

    props = dict(boxstyle='round', facecolor='white', alpha=0.5)
    arrow_props = dict(facecolor='black', shrink=0.1, width=1, headwidth=5, headlength=5)
    ax.annotate(f'R² = {r_squared:.2f}', xy=(popt[1], popt[2]), xytext=(popt[1] + 1, popt[2] + 1),
                arrowprops=arrow_props, bbox=props, fontsize=8)
plt.show()

Fitting successful for 2 spot(s) within blob 1!


In [17]:
# Determine and plot residual image, within found blobs...
fit_img = np.zeros_like(fixed_img)
contours = find_contours(blob_img, 1)

fit_img = PeakModel.multi_2d((x_coords.ravel(), y_coords.ravel()), *popt).reshape(*fixed_img.shape)

res_img = fixed_img * ~bkg_mask - fit_img * ~bkg_mask

fig, ax = plt.subplots(1, 3, figsize=(10, 3), dpi=200, sharex=True, sharey=True)
im = ax[0].imshow(fixed_img, extent=extent, vmin=0, vmax=100)
fig.colorbar(im, ax=ax[0])
ax[0].set_title('Experiment')

im = ax[1].imshow(fit_img, extent=extent, vmin=0, vmax=100)
fig.colorbar(im, ax=ax[1])
ax[1].set_title('Fit')

im = ax[2].imshow(res_img, extent=extent, vmin=-100, vmax=100, cmap='bwr', interpolation=None)
fig.colorbar(im, ax=ax[2])
ax[2].set_title('Residual')

for axi in ax:
    axi.plot(*blob_df['contour'][blob_num], c='r', lw=0.5)

plt.show()

In [642]:
correlations = []
for blob_num in range(len(blob_df)):    
    # Cross-correlation of blobs
    blob_bbox = [int(np.min(x_coords[blob_df['mask'][blob_num]])), int(np.max(x_coords[blob_df['mask'][blob_num]])) + 1,
                int(np.min(y_coords[blob_df['mask'][blob_num]])), int(np.max(y_coords[blob_df['mask'][blob_num]])) + 1]
    z_fit = blob_df['value'][blob_num][blob_bbox[2]:blob_bbox[3], blob_bbox[0]:blob_bbox[1]]
    #x_corr = correlate(fixed_img, z_fit, mode='same')
    x_corr = convolve2d(fixed_img, z_fit, mode='same')
    self_similar = peak_local_max(x_corr, min_distance=1, threshold_rel=0.65)

    for region in self_similar:
        for i, blob in enumerate(blob_df['mask']):
            if blob[*region]:
                correlations.append([blob_num, i])
                #print(f'Blob {blob_num} correlates with blob {i}')

# Probably a better way of doing this than a bunch of one line for loops...
uniq_corr, match_corr = [], []
[uniq_corr.append(x) for x in correlations if x not in uniq_corr and x[0] != x[1]];
[match_corr.append(x) for x in uniq_corr if x not in match_corr and x[::-1] not in match_corr and x[::-1] in uniq_corr];
[print(f'Blobs {x[0]} and {x[1]} might be from the same grain.') for x in match_corr];

In [643]:
blob_num = 1
# Cross-correlation of blobs
blob_bbox = [int(np.min(x_coords[blob_df['mask'][blob_num]])), int(np.max(x_coords[blob_df['mask'][blob_num]])) + 1,
             int(np.min(y_coords[blob_df['mask'][blob_num]])), int(np.max(y_coords[blob_df['mask'][blob_num]])) + 1]
z_fit = blob_df['value'][blob_num][blob_bbox[2]:blob_bbox[3], blob_bbox[0]:blob_bbox[1]]
x_corr = correlate(fixed_img, z_fit, mode='same')
#x_corr = convolve2d(fixed_img, z_fit, mode='same')
self_similar = peak_local_max(x_corr, min_distance=1, threshold_rel=0.65)

fig, ax = plt.subplots(1, 2, figsize=(10, 5), dpi=200, sharex=True, sharey=True)
ax[0].imshow(fixed_img)
ax[0].plot(blob_df['contour'][blob_num].T[1], blob_df['contour'][blob_num].T[0], c='r', lw=0.5)
for region in self_similar:
    ax[0].scatter(region[1], region[0], s=1, c='r')
ax[1].imshow(x_corr)
plt.show()

In [241]:
from skimage.feature import SIFT, match_descriptors

detector_extractor1 = SIFT()
detector_extractor2 = SIFT()
detector_extractor1.detect_and_extract(fixed_img)
detector_extractor2.detect_and_extract(z_fit)
matches = match_descriptors(detector_extractor1.descriptors,
                            detector_extractor2.descriptors,
                            max_ratio=np.inf)
print(matches)

[[262   0]
 [264   1]
 [268   2]
 [269   3]
 [272   6]
 [275   7]
 [279   9]
 [280   8]
 [572  11]
 [575  12]
 [783  13]
 [913  15]
 [950  14]]


In [242]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

ax.imshow(fixed_img, vmin=0)
points = detector_extractor1.keypoints[matches[:, 0]]
ax.scatter(points[:, 1], points[:, 0], s=1, c='r')
ax.plot(blob_df['contour'][blob_num].T[1], blob_df['contour'][blob_num].T[0], c='r', lw=0.5)
plt.show()

In [672]:
print(p0)
print(([0, 45, 45, 0, 0, -45, 0, 45, 25, 0, 0, -45,], [np.inf, 55, 55, np.inf, np.inf, 45, np.inf, 55, 35, np.inf, np.inf, 45,]))
print(generate_bounds(p0, Gaussian.func_2d))

[50, 50, 50, 5, 5, 0, 50, 50, 30, 5, 5, 0]
([0, 45, 45, 0, 0, -45, 0, 45, 25, 0, 0, -45], [inf, 55, 55, inf, inf, 45, inf, 55, 35, inf, inf, 45])
([0, 45.0, 45.0, 0, 0, -45, 0, 45.0, 45.0, 0, 0, -45], [inf, 55.0, 55.0, inf, inf, 45, inf, 55.0, 55.0, inf, inf, 45])


In [680]:
x_lst = np.linspace(0, 100, 100)
x_fit, y_fit = np.meshgrid(x_lst, x_lst)
x_fit = x_fit.ravel()
y_fit = y_fit.ravel()

#z_fit = Gaussian.func_2d((x_fit, y_fit), 50, 50, 50, 5, 5, 0)
p0 = [50, 50, 50, 5, 5, 0, 50, 50, 30, 5, 5, 0]
bounds = ([0, 45, 45, 0, 0, -45, 0, 45, 25, 0, 0, -45,], [np.inf, 55, 55, np.inf, np.inf, 45, np.inf, 55, 35, np.inf, np.inf, 45,])
bounds = generate_bounds(p0, Gaussian.func_2d)
z_fit = Gaussian.multi_2d((x_fit, y_fit), *p0) + (np.random.random(len(x_fit)) - 0.5) * 100
z_filt = gaussian_filter(z_fit, sigma=2)

fig, ax = plt.subplots(1, 2, figsize=(10, 5), dpi=200, sharex=True, sharey=True)
ax[0].imshow(z_fit.reshape((len(x_lst), len(x_lst))))
popt, pcov = curve_fit(Gaussian.multi_2d, (x_fit, y_fit), z_fit, p0=p0 + 5 * np.random.random(len(p0)), bounds=bounds)
z_fit = Gaussian.multi_2d((x_fit, y_fit), *popt)
ax[1].imshow(z_fit.reshape((len(x_lst), len(x_lst))))
plt.show()
print(*popt)

49.4821879058145 49.77467393135979 49.75974296037669 5.36264757762948 4.350332997984219 -20.89152706296693 41.15102361797082 50.4141744637802 30.136968607160178 5.74924925559718 6.50140961698663 25.008254047756036


In [641]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)
z_filt = gaussian_filter(z_fit.reshape((len(x_lst), len(x_lst))), sigma=5)
local_spots = peak_local_max(z_filt, min_distance=3, threshold_rel=0.65)
ax.imshow(z_filt)
ax.scatter(local_spots[:, 1], local_spots[:, 0], c='r', s=1)
plt.show()

In [678]:
generate_bounds([50, 30, 50, 5, 5, 0], Gaussian.func_2d)

([0, 25.0, 45.0, 0, 0, -45], [inf, 35.0, 55.0, inf, inf, 45])

In [438]:
zno_data = pd.read_csv(base + filedir + 'ZnO Integration Plot3.csv', delimiter=';')
x = zno_data['2th_deg'].tolist()
y = zno_data['intensity'].tolist()
background = restoration.rolling_ball(y, radius=1500)

fit_mask = (np.array(x) < 74.5) & (np.array(x) > 56)
x_data = np.array(x)[fit_mask]
y_data = np.array(y - background)[fit_mask]

x_fit = np.linspace(np.min(x_data), np.max(x_data), 5000)
    
# Normalize data
norm_int = 100 * (y_data - np.min(y_data)) / (np.max(y_data) - np.min(y_data))

# Find peak indices
peaks = find_peaks(norm_int, prominence=2, height=1, width=5)[0]
    
p_guess = []
for peak in peaks:
    p_guess.append(y_data[peak]) # a: amplitudes
    p_guess.append(x_data[peak]) # x0: peak centers
    p_guess.append(0.1) # sigma: peak width
    
#popt, pcov = curve_fit(all_peaks_fitting, x_data, y_data, p0=p_guess)
popt, pcov = curve_fit(Gaussian.multi_1d, x_data, y_data, p0=p_guess)


fig, ax = plt.subplots(2, 1, figsize=(9, 7), dpi=200, gridspec_kw={'height_ratios': [5, 1]})

ax[0].plot(x_fit, Gaussian.multi_1d(x_fit, *p_guess), '--', c='gray', label='Guess', lw=0.5)
ax[0].scatter(x_data, y_data, s=1, c='k', label='experiment')
ax[0].plot(x_fit, Gaussian.multi_1d(x_fit, *popt), c='r', label='Fit', lw=1)

ax[0].legend()

residuals =  y_data - Gaussian.multi_1d(x_data, *popt)
ss_res = np.sum(residuals ** 2)
ss_tot = np.sum(((y_data) - np.mean(y_data)) ** 2)
r_squared = 1 - (ss_res / ss_tot)

ax[1].set_title('Residuals')
ax[1].plot(x_data, residuals, c='r', lw=1)
ax[1].set_ylim(-1.15 * np.max(np.abs(residuals)), 1.15 * np.max(np.abs(residuals)))

plt.tight_layout()
print(f'R-squared is {r_squared:.4f}')

(1986,)
1986
('amp', 'x0', 'sigma')
37
dict_values([101.08437782167869, 57.20397, 0.1])
dict_values([41.88960107282807, 59.18836, 0.1])
dict_values([104.64633046617723, 59.72871, 0.1])
dict_values([9.929569739086727, 61.21001, 0.1])
dict_values([79.27510777407757, 62.47704, 0.1])
dict_values([85.56256245487863, 64.88066, 0.1])
dict_values([51.59800245487863, 66.85574, 0.1])
dict_values([5.533499739086729, 68.16934, 0.1])
dict_values([26.22182782167869, 71.11332, 0.1])
dict_values([2.907126111720121, 71.84931, 0.1])
dict_values([3.6235363009666024, 72.45487, 0.1])
dict_values([20.440600466177223, 73.64737, 0.1])
(1986,)
1986
('amp', 'x0', 'sigma')
37
dict_values([101.0843793279533, 57.20397, 0.1])
dict_values([41.88960107282807, 59.18836, 0.1])
dict_values([104.64633046617723, 59.72871, 0.1])
dict_values([9.929569739086727, 61.21001, 0.1])
dict_values([79.27510777407757, 62.47704, 0.1])
dict_values([85.56256245487863, 64.88066, 0.1])
dict_values([51.59800245487863, 66.85574, 0.1])
dict_

In [103]:
from math import sqrt
from skimage import data
from skimage.feature import blob_dog, blob_log, blob_doh
from skimage.color import rgb2gray

import matplotlib.pyplot as plt


image = test_img
image_gray = 1-test_img

blobs_log = blob_log(image_gray, max_sigma=30, num_sigma=10, threshold=.1)

# Compute radii in the 3rd column.
blobs_log[:, 2] = blobs_log[:, 2] * sqrt(2)

blobs_dog = blob_dog(image_gray, max_sigma=30, threshold=.1)
blobs_dog[:, 2] = blobs_dog[:, 2] * sqrt(2)

blobs_doh = blob_doh(image_gray, max_sigma=30, threshold=.01)

blobs_list = [blobs_log, blobs_dog, blobs_doh]
colors = ['yellow', 'lime', 'red']
titles = ['Laplacian of Gaussian', 'Difference of Gaussian',
          'Determinant of Hessian']
sequence = zip(blobs_list, colors, titles)

fig, axes = plt.subplots(1, 3, figsize=(9, 3), sharex=True, sharey=True)
ax = axes.ravel()

for idx, (blobs, color, title) in enumerate(sequence):
    ax[idx].set_title(title)
    ax[idx].imshow(image)
    for blob in blobs:
        y, x, r = blob
        c = plt.Circle((x, y), r, color=color, linewidth=2, fill=False)
        ax[idx].add_patch(c)
    ax[idx].set_axis_off()

plt.tight_layout()
plt.show()